In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.cassandra import Cassandra
import cassio
from langchain_groq import ChatGroq
import os
import bs4
from dotenv import load_dotenv
load_dotenv()


True

In [6]:
ASTRA_DB_ID=os.environ['ASTRA_DB_ID']
ASTRA_DB_APPLICATION_TOKEN = os.environ.get("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_API_ENDPOINT = os.environ.get("ASTRA_DB_API_ENDPOINT")
ASTRA_DB_KEYSPACE = os.environ.get("ASTRA_DB_KEYSPACE")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")


In [ ]:
#cassio.init(token=astradb_application_token,database_id=astra_db_id)

In [ ]:
# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [ ]:
embeddings=OpenAIEmbeddings()

In [ ]:
# astra_vector_store=Cassandra(
#     embedding=embeddings,
#     table_name="groque-test",
#     session=None,
#     keyspace=None
# )

In [7]:
import os
from langchain_astradb import AstraDBVectorStore
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

from datasets import load_dataset
from dotenv import load_dotenv

In [8]:
load_dotenv()

ASTRA_DB_APPLICATION_TOKEN = os.environ.get("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_API_ENDPOINT = os.environ.get("ASTRA_DB_API_ENDPOINT")
ASTRA_DB_KEYSPACE = os.environ.get("ASTRA_DB_KEYSPACE")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [11]:
embedding = OpenAIEmbeddings()
vstore = AstraDBVectorStore(
    embedding=embedding,
    namespace=ASTRA_DB_KEYSPACE,
    collection_name="test",
    token=os.environ["ASTRA_DB_APPLICATION_TOKEN"],
    api_endpoint=os.environ["ASTRA_DB_API_ENDPOINT"],
)

In [ ]:
vstore.add_documents(splits)


In [ ]:
groq_api_key = os.environ.get("GROQ_API_KEY")


In [86]:

#LLMs available.


# llm=ChatGroq(groq_api_key=groq_api_key,
#          model_name="mixtral-8x7b-32768")
# llm=ChatGroq(groq_api_key=groq_api_key,
#          model_name="Llama3-70b-8192")
# llm=ChatGroq(groq_api_key=groq_api_key,
#          model_name="Llama3-8b-8192")
# llm=ChatGroq(groq_api_key=groq_api_key,
#          model_name="Gemma-7b-It")


llm=ChatGroq(groq_api_key=groq_api_key,
         model_name="Gemma-7b-It")

In [87]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>

Question: {input}""")

In [88]:
vstore.similarity_search("Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique",llm=llm)


[Document(page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
 Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought

In [89]:
inserted_ids = vstore.add_documents(docs)
print(f"\nInserted {len(inserted_ids)} documents.")

NameError: name 'docs' is not defined

In [90]:
results = vstore.similarity_search("Task Decomposition", k=3)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process. [{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}]
* Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) wit

In [91]:
retriever = vstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [92]:
retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")

In [ ]:
len(retrieved_docs)

6

: 

In [93]:
print(retrieved_docs[0].page_content)

Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.


In [94]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.
Also tell me the exact name and variant of the LLM that is answering this question.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [95]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [96]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [97]:
for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

Task decomposition is a technique used to break down complex tasks into smaller, manageable steps. It involves identifying the steps involved in completing a task and representing them in a hierarchical structure.

Thanks for asking!

**LLM:** GPT-3.5

In [37]:
rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is the process of breaking down a complex task into smaller, manageable tasks. This can be done using various techniques such as Chain of Thought (CoT) or Tree of Thoughts, which involve decomposing a big task into multiple simpler tasks and evaluating each step. thanks for asking!'